In [1]:
import torch
import torch.nn as nn
from models.losses import TVLoss, FrequencyLoss, VGG19FeatureExtractor
from models.networks.blocks import ResnetBlock3D, ASPP3D, CBAM3D, PixelAwareAttention

def test_losses():
    print("Testing Losses...")
    # 模拟数据 [Batch, Channel, Depth, Height, Width]
    pred = torch.randn(2, 1, 16, 64, 64)
    target = torch.randn(2, 1, 16, 64, 64)
    
    # 1. TV Loss
    tv_loss = TVLoss()
    loss = tv_loss(pred)
    print(f"  TV Loss check: {loss.item():.4f} (Success)")
    
    # 2. FFL Loss
    ffl_loss = FrequencyLoss()
    loss = ffl_loss(pred, target)
    print(f"  FFL Loss check: {loss.item():.4f} (Success)")
    
    # 3. VGG (需要 RGB 3通道)
    vgg = VGG19FeatureExtractor()
    rgb_img = torch.randn(4, 3, 64, 64) # VGG处理的是2D切片
    feat = vgg(rgb_img)
    print(f"  VGG Feature shape: {feat.shape} (Success)")

def test_blocks():
    print("\nTesting Blocks...")
    x = torch.randn(2, 64, 16, 32, 32) # [B, C, D, H, W]
    norm = nn.BatchNorm3d
    
    # 1. ResnetBlock3D (With CBAM)
    block = ResnetBlock3D(dim=64, norm_layer=norm, use_dropout=False, use_bias=False)
    out = block(x)
    print(f"  ResnetBlock3D output: {out.shape} (Success)")
    
    # 2. ASPP3D
    aspp = ASPP3D(in_channels=64, out_channels=64, norm_layer=norm)
    out = aspp(x)
    print(f"  ASPP3D output: {out.shape} (Success)")
    
    # 3. PixelAwareAttention
    pa = PixelAwareAttention(in_channels=64, is_3d=True)
    out = pa(x)
    print(f"  PixelAwareAttention output: {out.shape} (Success)")

if __name__ == '__main__':
    try:
        test_losses()
        test_blocks()
        print("\n>>> Step 1 测试通过！模块拆分成功。")
    except Exception as e:
        print(f"\n>>> Step 1 测试失败！错误信息:\n{e}")
        exit(1)

Testing Losses...
  TV Loss check: 509.6630 (Success)
  FFL Loss check: 0.2763 (Success)


/home/liujia/miniconda3/envs/augan_wsl/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/liujia/miniconda3/envs/augan_wsl/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


  VGG Feature shape: torch.Size([4, 512, 4, 4]) (Success)

Testing Blocks...
  ResnetBlock3D output: torch.Size([2, 64, 16, 32, 32]) (Success)
  ASPP3D output: torch.Size([2, 64, 16, 32, 32]) (Success)
  PixelAwareAttention output: torch.Size([2, 64, 16, 32, 32]) (Success)

>>> Step 1 测试通过！模块拆分成功。


In [4]:
import torch
import torch.nn as nn
# 这里会测试 generator.py 是否正确引用了 blocks.py
from models.networks.generator import ResUnetGenerator

def test_generator():
    print("Testing ResUnetGenerator Linkage...")
    
    # 模拟参数
    input_nc = 1
    output_nc = 1
    ngf = 32
    norm_layer = nn.BatchNorm3d
    
    # 初始化模型 (这会触发内部所有 import)
    try:
        model = ResUnetGenerator(input_nc, output_nc, ngf, norm_layer=norm_layer, 
                                 n_blocks=6, use_aspp=True, use_attention=True)
        print("  Model initialized successfully.")
    except ImportError as e:
        print(f"  Import Error: {e}")
        return
    except Exception as e:
        print(f"  Other Error: {e}")
        return

    # 模拟前向传播
    x = torch.randn(1, 1, 128, 64, 64) # [B, C, D, H, W]
    try:
        out = model(x)
        print(f"  Forward pass output shape: {out.shape} (Success)")
    except Exception as e:
        print(f"  Forward pass Error: {e}")

if __name__ == '__main__':
    test_generator()

Testing ResUnetGenerator Linkage...
  Model initialized successfully.
  Forward pass Error: Expected more than 1 value per channel when training, got input size torch.Size([1, 256, 1, 1, 1])


In [10]:
import torch
import torch.nn as nn
from models.networks.generator import ResUnetGenerator

def test_generator():
    print("Testing ResUnetGenerator Linkage...")
    
    # 模拟参数
    input_nc = 1
    output_nc = 1
    ngf = 32
    # [修改] 保持使用 InstanceNorm3d
    norm_layer = nn.InstanceNorm3d 
    
    # 初始化模型
    try:
        # [修改] 将 n_blocks 从 6 改为 5，适配 64x64 的小尺寸输入
        model = ResUnetGenerator(input_nc, output_nc, ngf, norm_layer=norm_layer, 
                                 n_blocks=5, use_aspp=True, use_attention=True)
        print("  Model initialized successfully.")
    except Exception as e:
        print(f"  Model Init Error: {e}")
        return

    # 模拟前向传播
    x = torch.randn(1, 1, 128, 64, 64) 
    
    try:
        out = model(x)
        print(f"  Forward pass output shape: {out.shape} (Success)")
    except Exception as e:
        print(f"  Forward pass Error: {e}")

if __name__ == '__main__':
    test_generator()

Testing ResUnetGenerator Linkage...
  Model initialized successfully.
  Forward pass Error: Expected more than 1 spatial element when training, got input size torch.Size([1, 256, 1, 1, 1])


In [11]:
import torch
import torch.nn as nn
from models.networks.generator import ResUnetGenerator

def test_generator():
    print("1Testing ResUnetGenerator Linkage...")
    
    # 模拟参数
    input_nc = 1
    output_nc = 1
    ngf = 32
    # [修改] 保持使用 InstanceNorm3d
    norm_layer = nn.InstanceNorm3d 
    
    # 初始化模型
    try:
        # [修改] 将 n_blocks 从 6 改为 5，适配 64x64 的小尺寸输入
        model = ResUnetGenerator(input_nc, output_nc, ngf, norm_layer=norm_layer, 
                                 n_blocks=5, use_aspp=True, use_attention=True)
        print("  Model initialized successfully.")
    except Exception as e:
        print(f"  Model Init Error: {e}")
        return

    # 模拟前向传播
    x = torch.randn(1, 1, 128, 64, 64) 
    
    try:
        out = model(x)
        print(f"  Forward pass output shape: {out.shape} (Success)")
    except Exception as e:
        print(f"  Forward pass Error: {e}")

if __name__ == '__main__':
    test_generator()

1Testing ResUnetGenerator Linkage...
  Model initialized successfully.
  Forward pass Error: Expected more than 1 spatial element when training, got input size torch.Size([1, 256, 1, 1, 1])
